In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [ ]:
!git config --global user.email "zenglifan88@gmail.com"
!git config --global user.name "Lifan-Z"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('nferruz/ProtGPT2')
tokenizer = GPT2Tokenizer.from_pretrained('nferruz/ProtGPT2')

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/655k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token       # Cannot set: tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors = 'pt')

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader

remote_train = load_dataset("Lifan-Z/tox-antitox-proteins", split="train")
remote_validation = load_dataset("Lifan-Z/tox-antitox-proteins", split="validation")

# dataloader_train = DataLoader(remote_train, batch_size=4, collate_fn=data_collator, shuffle=True)
# dataloader_validation = DataLoader(remote_validation, batch_size=4, collate_fn=data_collator)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
print(remote_validation['input_ids'])

[[2038, 960, 3826, 25238, 271, 839, 2717, 442, 718, 14540, 580, 417, 30073, 41044, 559, 472, 49661, 3439, 8868, 199, 28585, 1793, 321, 1343, 1160, 1107, 1064, 654, 407, 10765, 507, 281, 23959, 2211, 318, 299, 789, 472, 682, 421, 7089, 199, 2356, 391, 9164, 6281, 793, 16871, 1536, 623, 414, 296, 1370, 30130, 1755, 620, 333, 499, 14217, 924, 271, 1068, 199, 2811, 2041, 21067, 1758, 3754, 43571, 430, 425, 716, 9168, 1509, 777, 356, 3623, 1088, 296, 13664, 2974, 54, 199, 356, 928, 6545, 370, 7358, 279, 3717, 833, 8686, 13023, 692, 7906, 316, 1254, 267, 2045, 3533, 27353, 7376, 199, 1088, 1556, 6264, 345, 18687, 2563, 438, 2855, 334, 21691, 799, 12506, 3838, 24197], [339, 915, 1463, 316, 3761, 335, 23592, 48301, 3401, 465, 5892, 453, 1719, 318, 579, 3076, 7707, 832, 1925, 319, 32930, 199, 3255, 982, 21987, 338, 1955, 835, 389, 27045, 3720, 25015, 1477, 849, 733, 612, 319, 22098, 1967, 371, 1749, 199, 16934, 1090, 448, 1336, 13247, 6228, 425, 356, 34550, 23290, 918, 492, 294, 861, 2957, 1006

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 8

args = TrainingArguments(
    output_dir="protGPT2_5",
    overwrite_output_dir=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    eval_steps=batch_size,
    logging_steps=batch_size,
    max_steps=5000,
    gradient_accumulation_steps=batch_size,
    # num_train_epochs=5,
    weight_decay=0.1,
    warmup_steps=4,
    lr_scheduler_type="cosine",
    learning_rate=1e-6,
    save_steps=396,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=remote_train,
    eval_dataset=remote_validation,
    # compute_metrics=calculatePerplexity,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,5.951300,5.712504
1,6.317100,5.708750
2,6.213600,5.705118
4,5.717500,5.699799
5,6.230400,5.697459
6,5.874600,5.695456
8,6.404900,5.691867
9,6.099300,5.690562
10,5.994200,5.689304
12,6.042400,5.687069


TrainOutput(global_step=5000, training_loss=5.761364625930786, metrics={'train_runtime': 3717.0957, 'train_samples_per_second': 21.522, 'train_steps_per_second': 1.345, 'total_flos': 6.57908969972736e+16, 'train_loss': 5.761364625930786, 'epoch': 202.02})

**-------------------------------------------------------------------------------------------------------------------------------**

###**Test the finetuned model**  

To prove that the model can generate sequences well.

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import pipeline

protgpt2 = pipeline('text-generation', model = 'Lifan-Z/protGPT2_5')

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [ ]:
sequences = protgpt2('<|endoftext|>', max_length=100, do_sample=True, top_k=950, repetition_penalty=1.2, num_return_sequences=10, eos_token_id=0)
for seq in sequences:
    print(seq)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'generated_text': '<|endoftext|>\nMLNISIKNLSYTYPGGTPVLRDVNFEFEHDKINWLICGKTGAGKSTFLEILSGLIKPDTG\nDVTINGRSLTSISVHDLRSRISIILQDTSLMSGSIYDNLLLPFRFLGLSEEQMRIKIEEV\nSRLLHIDHLLDQKVKVLSSGEFKRVVLARVLITNPRIILLDEPFSNIDIISKEKFEFLIK\nKWNREKTTVVFATHAQSDVMDPFVKIITLEKGKITQRDEKFSKYRFKSLHFDPVFNEDKK\nITRKEYIDYLKRSSLKFD'}
{'generated_text': '<|endoftext|>\nMQPTNLTFRAGQRWGWLILFVLIVVCAIFAAISTGLTVAFLAFFAINGLYHLIFYHSLRR\nEFEQHQPEITNQMSAVGHQRLGRYVFIALLVIGIGFLIFGPSFAKPIIIYTLGIMFLADG\nVARLIEAAYFFYFRGWIWPILDIGTGVFSLVLSSLFIGSLLTAIAIIIILVAIIAFYRGI\nLIAIHARRWKQTQHAPAQENNQPLVLQTPKQLPQRQTPQPQQPQQRTQLPQQQQRAPPQQ\nQMRPAQPPAP'}
{'generated_text': '<|endoftext|>\nMKRTDAVSFERTSRHKNQYQNGRKKRFQKTIKTISHDKSYFSRALFNIYWNAEGFRKALL\nYKMLDKGEWEKPFIIIQPLSDSAMYVLRRNKENSIVIEKGGKWNFLSSHFTYITEEEELK\nEYMQIQFKKNIPTAWIGKVLQINPINKKLYFAVYTFIEGKDFKNFEKEIKFNKESIFNLL\nQHLLTLYWELNKYGVDLLDQSMHNAILIDGQRKISFIDLDPIYLNIKEKRIQLYSKIKNF'}
{'generated_text': '<|endoftext|>\nMAAFQAGGAPEGIALAVGNLVHGKAAAGRRRDAGSGLADQAGAAQARTRKRGRIPRQRAT\nGRSDIEIARAAARAARGSG

**---------------------------------------------------------------------------------------------------------------------**

### **Analyze the function calculatePerplexity(sequence, model, tokenizer) in the paper**

To prove its result is similar **perplexity = evaluate.load("perplexity", module_type="metric")**

In [ ]:
import torch
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ", device)

device =  cpu


In [ ]:
# This function is from the paper.

def calculatePerplexity(sequence, model, tokenizer):
    input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0) # .unsqueeze(0):  tensor([..])-->tensor([[..]])
    input_ids = input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)

In [ ]:
# To analyze the function above line by line.
# The sequence is the second line below.

input_ids = torch.tensor(tokenizer.encode('MLRQLRPVDKSGKMGSLLWPLLFGASLALSPAIEPAWGQETSQNHGTASNVPLLSRLDAL\nEELRQALSLPTASIQTEAVTLDGRRLFLVAAPQGDPGSLPPVSQRARSIPNSAIRVVENL\nSNGWSRVDLGIVIAYENDLEQAIRITEQVALQMYREPAWRDKIREPPEMHGVDDLGERGI\nRLRIWIKVQPLQQWRVTREYRRRLKHAFDQAGIQIPFPQQALWFRSPLEMRIQGLSPEET\nHRLLHPMESRLSNPDPISEERAGQLPP')).unsqueeze(0)
input_ids

tensor([[  339,   970, 17566,   982,    43,  1661,   257,   712,  4537,   503,
          5282, 13228,  2163,   345, 37124,   366,   927,  1564,   570,   269,
           735,   199,    37,  5892,  2593, 17829,   387,  9900,  4729,   782,
           288,   550,    48,  1897,   296,  4260, 26745,   933, 39422,  4036,
           702,   287,   199, 23969,   552,   436, 11958, 11797,   448, 15731,
           278,  1310,   492,   603,   585,  1752, 12703, 41745,  2316,   298,
          3381,  3870,   428,  7755,   199, 25309,  3111,   362, 23255,   595,
          9768,  1676, 24025, 31031, 14673, 28228, 27147,   710,   334,   951,
         48299,  4231, 16438,   199,   524, 15280,  1794, 10863,  2113,  4170,
           888,  8338]])

In [ ]:
import torch

input_ids = input_ids.to(device)
model = model.to(device)           # Here model is "nferruz/ProtGPT2".
outputs = model(input_ids, labels=input_ids)
outputs

CausalLMOutputWithCrossAttentions(loss=tensor(7.8565, grad_fn=<NllLossBackward0>), logits=tensor([[[ -5.0986, -16.8763, -16.7865,  ...,  -3.2687,  -4.3534,  -4.8753],
         [  0.1206, -12.8329, -12.5444,  ...,  -1.0129,  -1.6504,  -1.4208],
         [ -3.2342, -16.8025, -16.4035,  ...,  -0.9820,   0.5041,   0.4691],
         ...,
         [  0.4460, -14.2469, -14.0690,  ...,  -1.4753,   0.1074,   0.5730],
         [ -0.2136, -14.1897, -13.9079,  ...,  -1.1533,   0.2446,   0.5600],
         [ -2.6222, -15.1465, -15.2052,  ...,  -1.4914,   0.1844,   0.7946]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-6.2751e-01,  5.1302e-01,  1.0940e+00,  ...,  3.5451e-01,
           -2.7793e-01,  1.4528e+00],
          [-1.4651e-01, -1.2534e-01, -4.2396e-01,  ..., -3.0860e-01,
            1.0931e-01, -2.1475e-01],
          [ 1.0109e+00,  1.3128e-01, -1.3776e+00,  ..., -2.4927e-01,
           -1.0137e+00,  6.1754e-01],
          ...,
          [-7.2769e-01,  1.7725e-01, -3

In [ ]:
loss, logits = outputs[:2]

In [ ]:
loss

tensor(7.8565, grad_fn=<NllLossBackward0>)

In [ ]:
logits

tensor([[[ -5.0986, -16.8763, -16.7865,  ...,  -3.2687,  -4.3534,  -4.8753],
         [  0.1206, -12.8329, -12.5444,  ...,  -1.0129,  -1.6504,  -1.4208],
         [ -3.2342, -16.8025, -16.4035,  ...,  -0.9820,   0.5041,   0.4691],
         ...,
         [  0.4460, -14.2469, -14.0690,  ...,  -1.4753,   0.1074,   0.5730],
         [ -0.2136, -14.1897, -13.9079,  ...,  -1.1533,   0.2446,   0.5600],
         [ -2.6222, -15.1465, -15.2052,  ...,  -1.4914,   0.1844,   0.7946]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
import math

math.exp(loss) #2nd

2582.392408963905

**----------------------------------------------------------------------------------------------------------------------------**

###**Compare perplexity**

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
input_texts = ['MASPELEVLGITTVAGNVSVEKTSRNARQICELAGCPQMAVYAGCPRPLLRPLQTAEEVH\nGKSGIDGANLPEPQMPLGSLHAVQYLIETLMAAQEPVTLALLGPMTNLAVALVQQPRIVE\nRIRRLVFMGGSAFEGNTTPAAEFNIFTDPHAAQIVLSAGIPEVVMLGLNVTQQVLSTPER\nIERIRALGTRVGQTVADMLAFYGKFDIRRYGLPGGPLHDPCVVAYLLQPQLFELKPCYVE\nVETASPLNLGRTVVDRWGLSGRPANVQVAFGVDAEEFYRLLTERLGRYR',
 'MLRQLRPVDKSGKMGSLLWPLLFGASLALSPAIEPAWGQETSQNHGTASNVPLLSRLDAL\nEELRQALSLPTASIQTEAVTLDGRRLFLVAAPQGDPGSLPPVSQRARSIPNSAIRVVENL\nSNGWSRVDLGIVIAYENDLEQAIRITEQVALQMYREPAWRDKIREPPEMHGVDDLGERGI\nRLRIWIKVQPLQQWRVTREYRRRLKHAFDQAGIQIPFPQQALWFRSPLEMRIQGLSPEET\nHRLLHPMESRLSNPDPISEERAGQLPP',
 'LVHGFGASIGHWRHNIPVLAAAGYRVYALDLLGFGGSAKPALAYSLELWAELLADFWRQH\nVRQPAVFIGNSIGALLSLLMAARYPQLVAGAVLLNCAGGLNHRSHELSPLARLFMATFTA\nLVASPVTGPLLFDWVRQRQRIRATLKQVYCNPAAITDELVEILYAPSCEPGAQQVFAAVL\nTAPPGPTPEALLPQVRCPLLVIWGERDPWTPIQRGRGFQNHVQGIDYQFFPLPDTGHCPH\nDERPEIVNPLILSWLPKAFATA',
 'MRGFSLWQSWKAAAVIASWGSLLLVADPVWAQLSDISTYWGAQYIAGLNERQIIGGFPDG\nTFRPNESITRAQFAVIVTKAFGLDTNVPVRPFADPIPSWAAPSIGAAAAAGLVSGFPDGT\nFRPNDVLTRAQAITVLTRAATNGRLLEDPGQIEAILSGFTDANLVPNFARAPIATAVQEG\nLLVLYPDPTRLNAQAVATRGEVAALTYQALAKVGRVPSLDPPVGAAIPGPGIRTELLAQV\nPPTPEPLATPDIPVEPLVETLPAAPEVRTFFAREELGSVKPGDTLTVYLLGSPGAQASFS\nIPGIAYNLPLQETRPGQYEGSYTFRSQDRASEAPVFARLERNGL',
 'MSIPAARQRFLQEIQADPIDLGRAALWIAQEAYPDLDVEEYLAALDEMAAEVQERLPPER\nYPLRVIRILNRYLFEDLGFCGNSEDYYDPRNSFLNEVIDRRTGIPITLSLIYLELARRID\nFPMAGVGMPGHFLVRPLFADAEIFVDPFHQGEILFPEDCQERLAQIYGPRIPLQDHHLRP\nTPPRLILVRLLNNLKQIYLSRAQLELALAAVERILLLIPESLPHLRDRGLLCYQLGRWQQ\nACQDLRHYLNQATLHPELSGPDEHLIQEILQRLEAYSD',
 'MGQRGSFISKDFPGAISDHLVFTRSFVNEHPEQVQAVVDAWFETTKFLETNPDEAIAIMA\nKKANVSVEEYKQYAGGTRIFTVQENITAFQPGNDMSSLPFAAKKISAFLTQVGLAKTQPD\nLSKLFDDRFVRAYAARVQTQTSPSTRNAHDRPRPFPAHS',
 'MFTNNEVWKTFLSNPVVFCGGFLSGLLRLDIRQDPLRSWLESQGLRVEDPPAPTQGPQSI\nSIE',
 'MRGIGLTEVYHISLCPAEEDSSLVKIANPLSPEYSAVRKALLPGLVEAFRFNWDQGNGPL\nQAFEIGHVFAYAPPSSPEPYQEAEHLGGILGGDLNPNDWRHHNRPMDWFEAKGLLVSALE\nRLGFTPEFRPDPPEAQSPLLHPGRQAGIWIEGSRLGLFGQLHPRLCQEKDLPEEVYGFEV\nDLDPLLASLEKRGLVQFSPFSPFPASDRDIAFFAPLELSVADIEKVIRQAAGPLLQSVQL\nFDEYRGQGVPPGQRSLAFRLVYRAPDRTLTDAEVEAAQNQVRAQLEKHFPVTLRS',
 'MSQRPEVVEGAEAKQKVTLYLPPELHRNLKIRAAVEREPMSAVAEKALTFYLLHPEVIEG\nVQGSAYRVHRCPECGHPFVIRNGEVVSLHQQSASEAILEEVEPEYWGAASASEEQRQDEL\nVPC',
 'MKAGWRSEETATLVSPSRFLSESTGVLPEKSQHNGAGVIQGAGVTPGGRSKVRIPSNLQF\nCRRFVFNHPTIISAGEIVMAVEKVNSSSSLAEVIDRILDKGIVVDAWVRVSLVGIELLAI\nEARVVVASVETYLKYAEAVGLTATAAAPAV',
 'VADLSDFIQGIEQLIELAQKLDEKKNTGGLRTEVRIQARSLGSSIPPSGSIPRRRSAQTG\nDPSWSSEPQVMSTSGWREPPKADPTPEATVADPPGSPDTWPGPRLKDVGGLKEQLQALRE\nLVEIPLKRPDLLAKLGLEPPRGVLLVGPPGTGKTLTARALAESLGVNYIALVGPELIGKY\nYGEAEARLRQVFEKAAKSAPCLVFIDEIDALVPNRAAVEGEVEKRLVAQMLGLMDGFVAQ\nKGVVVLAATNRPEALDPALRRPGRFDREVIFKVPDREGRREILAIHTRGMPLAEDVDLDS\nLADQTLGFVGADLRGLCQAAAYAALRRQVPDLGSPIPDSLTVSAADFQQALQQVKPAVLR\nSVEIESPQVSWDQIGGLEQAKQFLQEAIEGSLLHPELYEQAQAQAPKGILLSGPPGTGKT\nLLAKAIASQAKANFIAVSGPELLSKWVGSSEQAVRELFARARQCAPCVIFIDEIDTLAPA\nRGSYSGDSGVSDRVLGQLLAELDGIRPSQGVLVVAATNRKASLDPALTRAGRLELHLSVE\nLPDRAGRREILAVHNRKRPLGPDVDLEGWAERTEGWSGADLALLSNRAAIAAIRRHRATA\nESVDPKTLVIRQEDFAQAFAELQQQRQAT',
 'MFVNRLRSPFTPVGRLGIPAALGLLLLGILASSEVALAQRGDVNEAIRRDRNVQLFDGTS\nIDMNDLLRAAEFLSRQNAASRWDSKAGIDAAVEQFNRSRRPLVLSPRALTPESTPDP',
 'MLDVLLIVIFMLAGAGLGFYGLDLLPPAAIQGANVEGARVVTSGFGVLAGLAFGSLVRWG\nYRRFEANVRSLPPDVLISRSVGLVIGLVIANLAMAPVYLLPLPQQLNSFIEPLASVFFSL\nSFAYLGVSLADTHGPALLRLFNPNYALQAAAEGSVVAASPKVLDTSSIIDGRVAQLIETG\nFLEGTLVIPRFVIAELQTIADKADPQKRERGRRGLDTLQALRQAYPDRFVIHEANYPDLA\nTVDEKLVRLAQNVGGTLVTTDYNLKKVATLQGIKVLNVNELAEALRPIYIAGDSFDIKIT\nREGKEPGQGVGYLEDGTMVVVEEGQRSLGKTRSVVVTSAIQTSAGRMLFARLNKDSSPAS\nSAKT',
 'MNAFFRKLSALFAAYYAYMLEYRAELLLWALAGSLPLIMMGLWSEAARSGDFSLSSLEFI\nRYFLAVFLVRQFSIIWVIWDFEREVVEGRLSPLLLQPLDPAWHHVASHLMERLARLPFTV\nALIALFFLLYPRAFWIPSWQQVA',
 'VDEYQNQLEAFWTKPAAGLIRFDYETRQHGIQKVLTYPVCLFYARRAKYLAAFGQLPHAA\nRSLGWHNFRLDRIRSPRLTVVPWSDPAVPAELLARQRQGNLPTPEEVQAAWEEAWGSDFY\nLPKALLILRFAPQFARRYVDNTVRHPTFQPIAYERIPALVRREIADAGEREKILHILRQR\nPASDRYFCAYVRVGDTNFTMRLRDWRPNGEVIAPLAYREQMAQEAEAELRFYRPELALAK\nPACP',
 'VLIGSQHPVVVQSMINEDTLDIDAAVAAIRRLHEAGSEIVRVTVPSMAHAKAMEAIRSKL\nIQTYKPVPLVADVHHNGIKIALEVAQYVDKVRINPGLFVLEKPQPGRTEYTQAELEAIRN\nKIRETFTPLVQTLKAQNKALRIGVNHGSLAERMLFMYGDTPEGMVESALEYAEICAEQDF\nHNVVLSFKASRPQVMLAAYRLAARRFDALGLNYPFHLGVTEAGDGEYGRIKSAVGIGTLL\nAEGIGDTIRVSLTEAPEKEIPVAYGILQALNLRKTMVEYVACPSCGRTLFNLEEVLQKVR\nAATQHLVGLDIAVMGCIVNGPGEMADADYGYVGKTPGYISLYRGKEEVKKVPESEGVQAL\nIELIKADGRWVDPPEGSPSN',
 'MAEHIHRNWRITTQTTDKGFIFCCQRLRQDGSVEPAAKAYGYFQNEAEAILAGMVWIDRQ\nQLPDHLDGLLFDNEEPEG',
 'MAVAIAEKSLVAAFKQLGGPHHPPVIETFERGKTIFFPGDPAERVYFLVKGAVKLSRVYE\nMGEEITVALLRENSIFGVLSFITGQRSDRFYHAVAFTRVELLSLPIEQVEKSLRENAELS\nMLLLKSLSSRILQTEMMIETLAHRDMGSRLVSFLLILCRDFGVPGPNGVTIDLKLSHQAI\nAEAIGSTRVTITRLLGELRKKNYISIHKKKITVHDPMQLGHRFV',
 'MPSSAESLSSPLPQGEALKPSVVAAREPFLGLLRDLARAYQAFENYAGAHIRQMGLTAPQ\nFDVLVTLGNTPGLTMNVLAQKTLVTKGTLTGIVDRLEQKGLVRREVPPENRRCFKIVLTE\nEGEKLFEQVFPAHVAYLKERFQKLDPEEMAQIQACLKKLQSIFHLPRGQTQDC']

**Baseline:**

In [ ]:
import evaluate

perplexity = evaluate.load("perplexity", module_type="metric")

In [ ]:
results_baseline = perplexity.compute(model_id='nferruz/ProtGPT2', add_start_token=False, predictions=input_texts)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(list(results_baseline.keys()))

['perplexities', 'mean_perplexity']


In [ ]:
results_baseline["perplexities"] # The second output element is the same as calculatePerplexity()

[258.8265380859375,
 2582.29638671875,
 95.41284942626953,
 457.5437927246094,
 193.38182067871094,
 1557.81494140625,
 3315.23291015625,
 694.6611328125,
 2236.154541015625,
 462.505615234375,
 121.6801528930664,
 2299.309326171875,
 301.95501708984375,
 785.2081909179688,
 1266.2174072265625,
 69.1236572265625,
 6582.42236328125,
 555.4801025390625,
 434.6690673828125]

In [ ]:
results_baseline["mean_perplexity"]

1277.362937525699

**Finetuned model:**

In [ ]:
results_finetuned = perplexity.compute(model_id='Lifan-Z/protGPT2_5', add_start_token=False, predictions=input_texts)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
results_finetuned["perplexities"]

[255.1589813232422,
 2549.81298828125,
 89.08089447021484,
 457.11419677734375,
 191.29205322265625,
 1527.151611328125,
 3297.037109375,
 689.16845703125,
 2181.67578125,
 455.90106201171875,
 121.50672912597656,
 2279.6640625,
 300.7008056640625,
 769.6075439453125,
 1241.4920654296875,
 68.1622543334961,
 6443.55615234375,
 549.8592529296875,
 427.1698913574219]

In [ ]:
results_finetuned["mean_perplexity"]

891.5607131154914

**-----------------------------------------------------------------------------------------------------------**

**Also compare other fine tuned model:**

In [ ]:
results_finetuned_2 = perplexity.compute(model_id='Lifan-Z/protGPT2_2', add_start_token=False, predictions=input_texts)
results_finetuned_2["mean_perplexity"]

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

1257.6374680368524

In [ ]:
results_finetuned_3 = perplexity.compute(model_id='Lifan-Z/protGPT2_3', add_start_token=False, predictions=input_texts)
results_finetuned_3["mean_perplexity"]

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

1186.000286503842

In [ ]:
results_finetuned_4 = perplexity.compute(model_id='Lifan-Z/protGPT2_4', add_start_token=False, predictions=input_texts)
results_finetuned_4["mean_perplexity"]

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

1131.085124768709

###**Discussion**

The output "**mean_perplexity**" of the 5000-step finetuned model "**Lifan-Z/protGPT2_5**" is 891.56 .   

Compare with the baseline model "**nferruz/ProtGPT2**," its output "**mean_perplexity**" is 1277.36 .  

The finetuned model is much better.
